In [29]:
import os
from pathlib import Path
from tqdm import tqdm#_notebook as tqdm
from pku_autonomous_driving import io, util, dataset, resnet, centernet, prediction, graphics

import importlib
importlib.reload(io)
importlib.reload(util)
importlib.reload(dataset)
importlib.reload(resnet)
importlib.reload(centernet)
importlib.reload(prediction)
importlib.reload(graphics)

<module 'pku_autonomous_driving.graphics' from '/Users/argon/workspace/managed/pku-autonomous-driving/pku_autonomous_driving/graphics.py'>

In [30]:
import torch
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [31]:
import torchvision
from pku_autonomous_driving.transform import CropBottomHalf, PadByMean, Resize, Normalize, DropPointsAtOutOfScreen, CreateMask, CreateRegr, ToCHWOrder
from pku_autonomous_driving.const import IMG_WIDTH, IMG_HEIGHT, MODEL_SCALE

test_transform = torchvision.transforms.Compose([
    CropBottomHalf(),
    PadByMean(),
    Resize(IMG_WIDTH, IMG_HEIGHT),
    Normalize(),
    ToCHWOrder()
])

In [32]:
from pku_autonomous_driving.dataset import CarDataset, create_data_loader

test = io.load_test_data()

test_dataset = CarDataset(test, training=False, transform=test_transform)
test_loader = create_data_loader(test_dataset, batch_size=1)

In [33]:
base_model = resnet.resnext50_32x4d(pretrained=False)
model = centernet.CentResnet(base_model, 8)

setup_kwargs = {"model": model, "device": device}
if 'INITIAL_WEIGHTS' in os.environ:
    setup_kwargs["path"] = Path(os.environ["INITIAL_WEIGHTS"])
util.setup_model(**setup_kwargs)

{'model': CentResnet(
  (base_model): ResNet(
    (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
    (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (relu): ReLU(inplace=True)
    (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
    (layer1): Sequential(
      (0): Bottleneck(
        (conv1): Conv2d(64, 128, kernel_size=(1, 1), stride=(1, 1), bias=False)
        (bn1): BatchNorm2d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (conv2): Conv2d(128, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), groups=32, bias=False)
        (bn2): BatchNorm2d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (conv3): Conv2d(128, 256, kernel_size=(1, 1), stride=(1, 1), bias=False)
        (bn3): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (relu): ReLU(inplace=True)
        (do

In [34]:
%%time
predictions = prediction.predict(model, test_loader, device)

KeyboardInterrupt: 

In [7]:
test = io.load_test_data_raw()
test['PredictionString'] = predictions
test.to_csv('predictions.csv', index=False)
test.head()

NameError: name 'predictions' is not defined